# Sweeps - Eigenmode matrix

### Prerequisite
You need to have a working local installation of Ansys

## 1. Perform the necessary imports and create a QDesign in Metal first.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.analyses.sweep_options.sweeping import Sweeping

In [ ]:
# Create the design in Metal
# Create a design by specifying the chip size and open Metal GUI.

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

### In this example, the design consists of 1 qubit and 1 CPW connected to OpenToGround.

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

# Remove all qcomponents from GUI.
design.delete_all_components()

# So as to demonstrate the quality factor outputs easily, the
#subtrate material type is being changed to FR4_epoxy from the
#default of silicon
design.chips.main.material = 'FR4_epoxy'

q1 = TransmonPocket(
    design,
    'Q1',
    options=dict(pad_width='425 um',
                 pocket_height='650um',
                 connection_pads=dict(
                     readout=dict(loc_W=+1, loc_H=+1, pad_width='200um'))))
otg = OpenToGround(design,
                   'open_to_ground',
                   options=dict(pos_x='1.75mm', pos_y='0um', orientation='0'))
readout = RouteMeander(
    design, 'readout',
    Dict(
        total_length='6 mm',
        hfss_wire_bonds = True,
        fillet='90 um',
        lead=dict(start_straight='100um'),
        pin_inputs=Dict(start_pin=Dict(component='Q1', pin='readout'),
                        end_pin=Dict(component='open_to_ground', pin='open')),
    ))

gui.rebuild()
gui.autoscale()

## 2. Must open Ansys HFSS at this point.
### Registration occurs automatically within the method "sweep_one_option_get_eigenmode_solution_data". 

### Must have an open Ansys project and HFSS eigenmode design before executing render_design. Can do it manually or automate it.

Manually: To create a new Ansys project, click "New", the leftmost icon right below the file menu. 

Automate: Use the cells below to open the Ansys application.  The method sweep_one_option_get_eigenmode_solution_data will connect to ansys, activate a HFSS design, and add a setup.

In [ ]:
# This cell will open the Ansys application.  Run this cell, if Ansys, is not already open.
# A project will be opened using this command.
hfss = design.renderers.hfss
#hfss.open_ansys()

In [ ]:
### for render_design()
# Render every QComponent in QDesign.
render_qcomps = []

# Identify which kind of pins in Ansys. 
# Follow details from renderer in
# QHFSSRenderer.render_design.
# No pins are open, so don't need to utilize ender_endcaps.
render_endcaps = []

#List of tuples of jj's that shouldn't be rendered.  
#Follow details from renderer in QHFSSRenderer.render_design.
render_ignored_jjs = [('Q1','rect_jj')]

# Either calculate a bounding box based on the location of 
# rendered geometries or use chip size from design class.
render_box_plus_buffer = True



sweep = Sweeping(design)

In [ ]:
# Pass a dict with arguments to be used add Setup to project.
# If you don't pass all the arguments, the default is determined by 
# QHFSSRenderer's default_options.

# The name of setup will be "Sweep_em_setup".  
# If a setup named "Sweep_em_setup" exists in the project, it will be deleted, 
# and a new setup will be added with the arguments from setup_args.
em_setup_args = Dict(min_freq_ghz=None,
                  n_modes=None,
                  max_delta_f=None,
                  max_passes=None,
                  min_passes=None,
                  min_converged=None,
                  pct_refinement=None,
                  basis_order=None)

# Example of updating the keys that we are interested in. 
em_setup_args = Dict(min_freq_ghz=5,
                  n_modes=2,
                  max_passes=9,)

## 3 Metal pass information to Ansys, and gets a solution matrix.
### - Connect to Ansys HFSS, eigenmode solution.
### - Rebuild QComponents in Metal.
### - Render QComponents within HFSS and setup.
### - Delete/Clear the HFSS between each calculation of solution matrix.
### - Calculate solution matrix for each value in option_sweep.

#### Return a dict and return code.  If the return code is zero, there were no errors detected.  
#### The dict has:  key = each value used to sweep, value = Dict with frequency, kappa_over_2pis, and quality_factor.

#### This could take minutes based size of design.

In [ ]:
#The argument "leave_last_design" is true by default.  

#Note: The method will connect to Ansys, activate_eigenmode_design(), add_eigenmode_setup().

all_sweeps, return_code = sweep.sweep_one_option_get_eigenmode_solution_data(
    readout.name,
    'total_length', 
    ['5.8mm', '6mm', '6.2mm'],
    render_qcomps,
    render_endcaps,
    render_ignored_jjs,
    box_plus_buffer_render=render_box_plus_buffer,
    design_name="GetEigenModeSolution",
    setup_args=em_setup_args)


In [ ]:
if return_code ==0:
    print(f'{all_sweeps.keys()} \n')
    for key in all_sweeps.keys():
        print(f'\nkey={key}')
        option_name = all_sweeps[key]['option_name']
        print(f'option_name[\'{key}\'][\'option_name\']={option_name}')
        did_converge = all_sweeps[key]['convergence']
        print(f'convergence: {did_converge}')
        print('frequency:')
        print(all_sweeps[key]['frequency'])
        print('kappa_over_2pis:')
        print(all_sweeps[key]['kappa_over_2pis'])
        print('quality_factor:')
        print(all_sweeps[key]['quality_factor'])
        print('convergence_t:\n')
        print(all_sweeps[key]['convergence_t'])
        print('convergence_eig_f:\n')
        print(all_sweeps[key]['convergence_eig_f'])
     

In [ ]:
# For example, just one group of solution data.
all_sweeps['6mm']

In [ ]:
# In HFSS, after the last sweep, should the design be cleared?  The argument "leave_last_design" is true by default. 
# If you want to delete after last sweep, change argument to false.

leave_last_design = False

all_sweeps, return_code = sweep.sweep_one_option_get_eigenmode_solution_data(
                                q1.name,
                                'connection_pads.readout.pad_width', 
                                ['100um', '110um', '120um'],
                                render_qcomps,
                                render_endcaps,
                                render_ignored_jjs,
                                box_plus_buffer_render=render_box_plus_buffer,
                                design_name="GetEigenModeSolution",
                                leave_last_design=leave_last_design,
                                setup_args=em_setup_args)

In [ ]:
# Uncomment next line if you would like to close the gui
# gui.main_window.close()